In [1]:
import nltk
nltk.download('webtext')
from nltk.corpus import webtext

[nltk_data] Downloading package webtext to /Users/ivan/nltk_data...
[nltk_data]   Package webtext is already up-to-date!


In [2]:
#first ;et's check how oir corpus Look like
words = [w.lower() for w in webtext.words('grail.txt')]
output = ' '.join(words)
print(output[:200])

scene 1 : [ wind ] [ clop clop clop ] king arthur : whoa there ! [ clop clop clop ] soldier # 1 : halt ! who goes there ? arthur : it is i , arthur , son of uther pendragon , from the castle of camelo


In [5]:
print(len(words))
print(len(output.split()))
print(len(output))


16967
16967
69827


In [8]:
from nltk.corpus import stopwords
stopwords=stopwords.words('english')

In [9]:
without_stopwords = [ x for x in words if x not in stopwords]
print(len(without_stopwords))

12209


In [11]:
no_punctutation_words = [x for x in without_stopwords if x.isalnum()]
print(len(no_punctutation_words))

6844


#  
#  
# Colocations
we extract them

In [12]:
# Bigram
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
# words= [w.lower() for w in webtext.words('grail.txt')]

bcf= BigramCollocationFinder.from_words(words)
print(bcf.nbest(BigramAssocMeasures.likelihood_ratio, 10)) #only prints 10 combinations



[("'", 's'), ('arthur', ':'), ('#', '1'), ("'", 't'), ('villager', '#'), ('#', '2'), (']', '['), ('1', ':'), ('oh', ','), ('black', 'knight')]


### Challenge: that output is not what is expected, discuss reasons
- do some cleaning of stop words
- and some filter, like >len(3)

### Bigram

In [15]:
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
# words= [w.lower() for w in webtext.words('grail.txt')]

#Filter
filter_stop = lambda w:len(w)<3 or w in stopwords
bcf= BigramCollocationFinder.from_words(words)
bcf.apply_word_filter(filter_stop)
#Frequency
# Let's only keep the bigrams that are repeated minimum of 'n' times
n=20
bcf.apply_freq_filter(n)

#Print the matches
print(bcf.nbest(BigramAssocMeasures.likelihood_ratio, 20)) 

[('black', 'knight'), ('clop', 'clop'), ('head', 'knight'), ('mumble', 'mumble'), ('french', 'guard')]


### Trigram

In [20]:
from nltk.collocations import TrigramCollocationFinder
from nltk.metrics import TrigramAssocMeasures
# words= [w.lower() for w in webtext.words('grail.txt')]
# freq=5
def get_trigram(freq):
    #Filter
    filter_stop = lambda w:len(w)<3 or w in stopwords
    tcf= TrigramCollocationFinder.from_words(words)
    tcf.apply_word_filter(filter_stop)
    #Frequency
    tcf.apply_freq_filter(freq)
    #Print the matches
    print(tcf.nbest(TrigramAssocMeasures.likelihood_ratio, 20)) 
    print()
    
get_trigram(5)
get_trigram(3)


[('clop', 'clop', 'clop'), ('mumble', 'mumble', 'mumble'), ('squeak', 'squeak', 'squeak'), ('saw', 'saw', 'saw'), ('pie', 'iesu', 'domine'), ('clap', 'clap', 'clap'), ('dona', 'eis', 'requiem'), ('brave', 'sir', 'robin'), ('heh', 'heh', 'heh'), ('king', 'arthur', 'music'), ('boom', 'boom', 'boom')]

[('clop', 'clop', 'clop'), ('mumble', 'mumble', 'mumble'), ('squeak', 'squeak', 'squeak'), ('saw', 'saw', 'saw'), ('pie', 'iesu', 'domine'), ('clap', 'clap', 'clap'), ('dona', 'eis', 'requiem'), ('brave', 'sir', 'robin'), ('heh', 'heh', 'heh'), ('king', 'arthur', 'music'), ('hee', 'hee', 'hee'), ('holy', 'hand', 'grenade'), ('boom', 'boom', 'boom'), ('...', 'dona', 'eis'), ('already', 'got', 'one'), ('good', 'sir', 'knight')]



In [21]:
# Verify the times of repetition
import re
print(len(re.findall('clop clop clop', output)))

13


In [22]:
# count the frequenc of each trigram

for k,v in tcf.ngram_fd.items():
    print(k,v)

('clop', 'clop', 'clop') 13
('king', 'arthur', 'music') 9
('pie', 'iesu', 'domine') 10
('dona', 'eis', 'requiem') 7
('clap', 'clap', 'clap') 5
('saw', 'saw', 'saw') 14
('squeak', 'squeak', 'squeak') 15
('brave', 'sir', 'robin') 8
('heh', 'heh', 'heh') 5
('mumble', 'mumble', 'mumble') 10
('boom', 'boom', 'boom') 5


In [23]:
for k,v in bcf.ngram_fd.items():
    print(k,v)

('clop', 'clop') 26
('black', 'knight') 33
('french', 'guard') 21
('head', 'knight') 29
('mumble', 'mumble') 20


# PMI Pointwise mutual information
the value if it is more
larger, the higher 
smaller, the value, is more repeated and is more likely 

In [28]:
from nltk.collocations import *
from nltk.tokenize import word_tokenize
text = "this is a foo bar bar black sheep foo bar bar black sheep foo bar bar black sheep shep bar bar black sentence"
trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder = TrigramCollocationFinder.from_words(word_tokenize(text))
for i in finder.score_ngrams(trigram_measures.pmi):
    print(i)

(('this', 'is', 'a'), 9.047123912114026)
(('is', 'a', 'foo'), 7.46216141139287)
(('black', 'sheep', 'shep'), 5.46216141139287)
(('black', 'sheep', 'foo'), 4.877198910671714)
(('a', 'foo', 'bar'), 4.46216141139287)
(('sheep', 'shep', 'bar'), 4.46216141139287)
(('bar', 'black', 'sentence'), 4.047123912114026)
(('bar', 'black', 'sheep'), 4.047123912114026)
(('sheep', 'foo', 'bar'), 3.877198910671714)
(('bar', 'bar', 'black'), 3.047123912114026)
(('foo', 'bar', 'bar'), 3.047123912114026)
(('shep', 'bar', 'bar'), 3.047123912114026)


In [35]:
#first ;et's check how oir corpus Look like
words_singles = [w.lower() for w in webtext.words('singles.txt')]
cleaner_singles = [w for w in words_singles if w not in stopwords and w.isalnum()]
output_singles = ' '.join(cleaner_singles)
print(output_singles[:200])

25 sexy male seeks attrac older single lady discreet encounters 35yo security guard seeking lady uniform fun times 40 yo single dad sincere friendly dte seeks r ship fem age open e 44yo tall seeks wor


In [36]:
trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder = TrigramCollocationFinder.from_words(word_tokenize(output_singles))
for i in finder.score_ngrams(trigram_measures.pmi):
    print(i)

(('35yo', 'security', 'guard'), 22.961580402191633)
(('advertise', 'perfect', 'match'), 22.961580402191633)
(('available', 'day', 'appreciate'), 22.961580402191633)
(('charters', 'towers', 'especially'), 22.961580402191633)
(('coastal', 'town', 'eyre'), 22.961580402191633)
(('dog', 'thinks', 'human'), 22.961580402191633)
(('dont', 'forget', 'free'), 22.961580402191633)
(('feisty', 'foxy', 'yummy'), 22.961580402191633)
(('forget', 'free', 'advertise'), 22.961580402191633)
(('foxy', 'yummy', 'grandmummy'), 22.961580402191633)
(('free', 'advertise', 'perfect'), 22.961580402191633)
(('fulfillment', 'alone', 'lost'), 22.961580402191633)
(('glorious', 'sunsets', 'candlelight'), 22.961580402191633)
(('golf', 'bike', 'riding'), 22.961580402191633)
(('heading', 'darwin', 'may'), 22.961580402191633)
(('headmaster', 'firm', 'hand'), 22.961580402191633)
(('important', 'distance', 'speed'), 22.961580402191633)
(('js', 'bayswater', 'mine'), 22.961580402191633)
(('lationship', 'coastal', 'town'), 22.

# 
# 
# WORDNET
## synonym and antonym

Word.POS.sensNumber?


In [37]:
from nltk.corpus import wordnet

In [39]:
syn = wordnet.synsets('cat')
print(len(syn))
print(syn)


10
[Synset('cat.n.01'), Synset('guy.n.01'), Synset('cat.n.03'), Synset('kat.n.01'), Synset('cat-o'-nine-tails.n.01'), Synset('caterpillar.n.02'), Synset('big_cat.n.01'), Synset('computerized_tomography.n.01'), Synset('cat.v.01'), Synset('vomit.v.01')]


In [44]:
syn = wordnet.synsets('spring')
print(len(syn))
print(syn)

11
[Synset('spring.n.01'), Synset('spring.n.02'), Synset('spring.n.03'), Synset('spring.n.04'), Synset('give.n.01'), Synset('leap.n.01'), Synset('jump.v.01'), Synset('form.v.03'), Synset('bounce.v.01'), Synset('spring.v.04'), Synset('spring.v.05')]


### Get the synonym and antonym of specific word

In [45]:
# import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')
from nltk.corpus import wordnet


[nltk_data] Downloading package omw-1.4 to /Users/ivan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/ivan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [53]:
def get_syn_info(_word):
    synonyms = []
    antonyms = []
    input_word = _word
    for syn in wordnet.synsets(input_word):
        for l in syn.lemmas():
            synonyms.append(l.name())
            if l.antonyms():
                antonyms.append(l.antonyms()[0].name())
    print(_word)
    print(set(synonyms))
    print(set(antonyms))
    print("")
    
_words=['good','mint','nice',]
# _words=['thoroughly','serious','unspoilt']
for w in _words:
    get_syn_info(w)


good
{'dear', 'estimable', 'skilful', 'honorable', 'undecomposed', 'soundly', 'upright', 'effective', 'sound', 'in_force', 'secure', 'salutary', 'ripe', 'full', 'unspoilt', 'serious', 'right', 'commodity', 'respectable', 'proficient', 'well', 'thoroughly', 'unspoiled', 'trade_good', 'dependable', 'near', 'in_effect', 'honest', 'safe', 'goodness', 'just', 'beneficial', 'practiced', 'adept', 'skillful', 'good', 'expert'}
{'bad', 'badness', 'evilness', 'ill', 'evil'}

mint
{'mass', 'stack', 'plenty', 'mint', 'deal', 'lot', 'quite_a_little', 'batch', 'pile', 'spate', 'muckle', 'sight', 'tidy_sum', 'mint_candy', 'great_deal', 'slew', 'flock', 'mountain', 'wad', 'mickle', 'mess', 'peck', 'heap', 'passel', 'raft', 'pot', 'good_deal', 'coin', 'strike', 'hatful'}
set()

nice
{'decent', 'squeamish', 'gracious', 'overnice', 'prissy', 'skillful', 'dainty', 'Nice', 'courteous', 'nice'}
{'nasty'}



## Example 2

In [ ]:
from nltk.corpus import wordnet


In [56]:
syns = wordnet.synsets('jump')
print('1-\nsyns:', syns)

#An example of a synset
print("2-\n",syns[0].name())

#jsut the word
print("3-\n",syns[0].lemmas()[0].name())

#Definition of that first synset
print('4-\nDefinition of hte word:', syns[0].definition())

# Examples of the word in use in sentences:
print('5-\nExamples of the word in use in sentences:', syns[0].examples())

1-
syns: [Synset('jump.n.01'), Synset('leap.n.02'), Synset('jump.n.03'), Synset('startle.n.01'), Synset('jump.n.05'), Synset('jump.n.06'), Synset('jump.v.01'), Synset('startle.v.02'), Synset('jump.v.03'), Synset('jump.v.04'), Synset('leap_out.v.01'), Synset('jump.v.06'), Synset('rise.v.11'), Synset('jump.v.08'), Synset('derail.v.02'), Synset('chute.v.01'), Synset('jump.v.11'), Synset('jumpstart.v.01'), Synset('jump.v.13'), Synset('leap.v.02'), Synset('alternate.v.01')]
2-
 jump.n.01
3-
 jump
4-
Definition of hte word: a sudden and decisive increase
5-
Examples of the word in use in sentences: ['a jump in attendance']


In [61]:
def get_more_syn_info(_word):
    syns=wordnet.synsets(_word)
    print('---------------\n',_word)
    print('1:\t',syns)
    print('2:\t',syns[0].name())
    print('3:\t',syns[0].lemmas()[0].name())
    print('4:\tDefinition:\t', syns[0].definition())
    print('5:\tExamples:\t',syns[0].examples())
    
    

In [62]:
get_more_syn_info('joy')
get_more_syn_info('supreme')
get_more_syn_info('starvation')

---------------
 joy
1:	 [Synset('joy.n.01'), Synset('joy.n.02'), Synset('rejoice.v.01'), Synset('gladden.v.01')]
2:	 joy.n.01
3:	 joy
4:	Definition:	 the emotion of great happiness
5:	Examples:	 []
---------------
 supreme
1:	 [Synset('supreme.s.01'), Synset('sovereign.s.02'), Synset('supreme.s.03'), Synset('supreme.s.04')]
2:	 supreme.s.01
3:	 supreme
4:	Definition:	 final or last in your life or progress
5:	Examples:	 ['the supreme sacrifice', 'the supreme judgment']
---------------
 starvation
1:	 [Synset('starvation.n.01'), Synset('starvation.n.02')]
2:	 starvation.n.01
3:	 starvation
4:	Definition:	 a state of extreme hunger resulting from lack of essential nutrients over a prolonged period
5:	Examples:	 []


#  
#  
# Similarity between words

In [65]:
# import nltk
# from nltk.corpus import wordnet
#compare ship and boat
w1 = wordnet.synset('ship.n.01')
w2 = wordnet.synset('boat.n.01')
print(w1.wup_similarity(w2))

0.9090909090909091


#  
#  
# Re.Compile

In [69]:
import re

str1 = "Emma's luck numbers are 251 761 231 451"

string_pattern = r'\d{3}'
regex_pattern = re.compile(string_pattern)

print(type(regex_pattern))


<class 're.Pattern'>


In [72]:
result = regex_pattern.findall(str1)
print('output from str1: ', result)

str2 = "Kelly's 33 lucky numbers are 111 212 415 12124"

result = regex_pattern.findall(str2)
print('output from str2:   ',result)

output from str1:  ['251', '761', '231', '451']
output from str2:    ['111', '212', '415', '121']


## Word lenghtening
using compile, substitute,  
to "maintian" the variable extracted, is \1 or \2 depending the pattern used

In [86]:
import re

def remove_lengthening(text):
    # \1: Backreference to the first captured group, which represents the character matched by (.).
    patt=re.compile(r'(.)\1{1,}') # find this pattern 
    return patt.sub(r'\1\1', text) # find the patters and replace it in the text

print(remove_lengthening('booooooooook'))



book


In [89]:
import re

def remove_lengthening(text):
    pattern = re.compile(r'(?<=(.))\1+')
    return pattern.sub(r'\1\1', text)

print(remove_lengthening('booooooooook'))


boook


In [90]:
import re

def remove_lengthening(text):
    pattern = re.compile(r'(\w)\1+')
    return pattern.sub(r'\1\1', text)

print(remove_lengthening('booooooooook'))


book
